In [133]:
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
import re
#import logging
#import json
#import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

In [2]:
url='url'
filepath='C:/Users/Neon/Desktop/GitHub/download_entsoe/2017_2_AggregatedGenerationPerType.csv'
headers = ['region', 'variable', 'attribute', 'source', 'subsource','region_type','web']

In [18]:
def read_entsoe_transparency(filepath, url, headers):
    df = pd.read_csv(
        filepath,
        sep='\t',
        encoding='utf-16',
        header=0,
        index_col=['DateTime'],
        parse_dates=True,
        converters=[{'MapCode': lambda x: x[:2]},
                    {'AreaName': lambda x: x.replace(' CA','')},
                    {'AreaName': lambda x: x.replace(' BZ','')}],
        nrows=20,
        )
    
    df.drop(['year', 'month', 'day', 'AreaTypeCode', 'SubmissionTS'], 
            axis=1, inplace=True)
    df.rename(columns ={
        'ActualConsumption': 'consumption', 
        'ActualGenerationOutput': 'generation'},inplace=True)
    df.set_index(
        ['MapCode','ProductionType_Name','AreaName','AreaTypeCode'],
        append=True, inplace=True)
    df=df.unstack('MapCode')
    df=df.unstack('ProductionType_Name')
    df=df.unstack('AreaName')
    df=df.unstack('AreaTypeCode')
    df=pd.concat(
        [df], 
        keys=['ENTSO-E'], 
        names=['source', 'attribute', 'region', 'variable', 'subsource'],
        axis=1)
    df=pd.concat([df], keys= url, names=['web'], axis=1)
    df=df.reorder_levels(headers, axis=1)
    
    return df

In [152]:
df = pd.read_csv(
    filepath,
    sep='\t',
    encoding='utf-16',
    header=0,
    index_col=['DateTime'],
    parse_dates=True,
    converters={'MapCode': lambda y: y[:2],
                #'AreaName': (lambda x: x.replace(' CA','')),
                #'AreaName': map(lambda x: x.replace(' CA','').rstrip(' BZ')),
                #'AreaName': lambda z: z.replace(' BZ',''),
                },
    nrows=20,
    )

#df['AreaName'].replace(' CA','',inplace=True),
#df['AreaName'].re.sub(' CA',''),
#df['AreaName'] = df['AreaName'].map(lambda x: x.rstrip(' CA').rstrip(' BZ')),
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' CA','')),
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' BZ','')),
df['AreaName']=[x.replace(' CA','').replace(' BZ','') for x in list(df['AreaName'])],

df.drop(['year', 'month', 'day'], 
        axis=1, inplace=True)
df.rename(columns ={
    'ActualConsumption': 'consumption', 
    'ActualGenerationOutput': 'generation'},inplace=True)
df.set_index(
    ['MapCode','ProductionType_Name','AreaName','AreaTypeCode'],
    append=True, inplace=True)
df=df.unstack('MapCode')
df=df.unstack('ProductionType_Name')
df=df.unstack('AreaName')
df=df.unstack('AreaTypeCode')
df=pd.concat(
    [df], 
    keys=['ENTSO-E'], 
    names=['source', 'attribute', 'region', 'variable', 'subsource','region_type'],
    axis=1)
df=pd.concat([df], keys= url, names=['web'], axis=1)
df=df.reorder_levels(headers, axis=1)

ValueError: Length of values does not match length of index

In [153]:
df = pd.read_csv(
    filepath,
    sep='\t',
    encoding='utf-16',
    header=0,
    index_col=['DateTime'],
    parse_dates=True,
    converters={#'MapCode': lambda y: y[:2],
                #'AreaName': (lambda x: x.replace(' CA','')),
                #'AreaName': map(lambda x: x.replace(' CA','').rstrip(' BZ')),
                #'AreaName': lambda z: z.replace(' BZ',''),
                #'AreaName': map(lambda x: x.replace(' CA','').replace(' BZ',''))
                },
    nrows=500,
    )

In [145]:
#df['AreaName'].replace(' CA','',inplace=True),
#df['AreaName']= list(df['AreaName'].map(lambda x: x.replace(' CA','').replace(' BZ',''))),
#df['AreaName'].apply(lambda x: x.rstrip(' CA')),
#len(df['AreaName'].map(lambda x: x.replace(' CA',''))),
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' BZ','')),
#df['MapCode'] = df['MapCode'].map(lambda y: y[:2]),
#df['MapCode'] = [w.replace(' CA','') for w in df['MapCode']],
#df['MapCode'] = map(lambda x: str.replace(x, ' CA',''), list(df['MapCode']))

(500,)

In [143]:
#s=list(s)
#len(df['AreaName'].map(lambda x: re.findall(" CA", x)))
#len(df['AreaName'].index)

500

In [171]:
df['AreaName']=[x.replace(' CA','').replace(' BZ','') for x in list(df['AreaName'])],

ValueError: Length of values does not match length of index

In [159]:
#df.head()
df['AreaName']

DateTime
2017-02-01 12:15:00         MAVIR CA
2017-02-01 12:15:00         MAVIR CA
2017-02-01 18:15:00         MAVIR CA
2017-02-01 10:00:00         MAVIR CA
2017-02-01 11:45:00         MAVIR CA
2017-02-23 03:00:00    TransnetBW CA
2017-02-23 15:00:00    TransnetBW CA
2017-02-23 16:45:00    TransnetBW CA
2017-02-23 09:15:00    TransnetBW CA
2017-02-23 00:30:00    TransnetBW CA
2017-02-23 01:30:00    TransnetBW CA
2017-02-23 16:30:00    TransnetBW CA
2017-02-10 09:30:00    TransnetBW CA
2017-02-10 04:45:00    TransnetBW CA
2017-02-10 15:45:00    TransnetBW CA
2017-02-10 18:00:00    TransnetBW CA
2017-02-10 01:00:00    TransnetBW CA
2017-02-10 01:00:00    TransnetBW CA
2017-02-10 22:00:00    TransnetBW CA
2017-02-10 17:30:00    TransnetBW CA
2017-02-10 01:45:00    TransnetBW CA
2017-02-10 06:00:00    TransnetBW CA
2017-02-10 01:30:00    TransnetBW CA
2017-02-10 07:15:00    TransnetBW CA
2017-02-10 14:00:00    TransnetBW CA
2017-02-10 06:30:00    TransnetBW CA
2017-02-09 02:00:00    Transn